In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import copy

In [2]:
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
le = LabelEncoder()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
df = data.copy()
cat_feat = list(df.dtypes[df.dtypes == object].index)
for column in cat_feat:
    df = df[df[column] != '?']
df.shape

(45222, 15)

In [6]:
le.fit( df['income'] )
y = pd.Series(data = le.transform(df['income']))
df.drop('income', axis = 1, inplace= True)
cat_feat.remove('income')

In [7]:
le.transform(['<=50K', '>50K'])

array([0, 1], dtype=int64)

Целевые переменные: 0 - <=50K, 1 - >50K

In [8]:
num_feat = [f for f in df if f not in (cat_feat)]

In [9]:
cat_nunique = df[cat_feat].nunique()
num_feat = [f for f in df if f not in (cat_feat + ['ID', 'target'])]
print(cat_nunique)
cat_feat = list(cat_nunique[cat_nunique < 30].index)

workclass          7
education         16
marital-status     7
occupation        14
relationship       6
race               5
gender             2
native-country    41
dtype: int64


In [10]:
dummy_train = pd.get_dummies(df[cat_feat], columns=cat_feat)


In [11]:
X_train = pd.concat([df[num_feat].fillna(-999),
                     dummy_train], axis=1)

In [12]:
X_train.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male
0,25,226802,7,0,0,40,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
1,38,89814,9,0,0,50,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,44,160323,10,7688,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
5,34,198693,6,0,0,30,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

In [14]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3)

for train_index, test_index in splitter.split(X_train, y):
    d_train = X_train.iloc[train_index]
    d_test = X_train.iloc[test_index]
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

In [15]:
logr = LogisticRegression()

In [16]:
logr.fit(d_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
logr.score(d_test, y_test)

0.7915530330950099